# REVIEWS SCRAPING

In [3]:
#libraries needed
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

import time
from parsel import Selector

import pandas as pd

from google_play_scraper import search
from google_play_scraper import Sort, reviews

from bs4 import BeautifulSoup

* Download applyzer.com page source (applyzer is a website that daily updates the play store apps ranking)

* Use selenium since the page source has different java script fucntions, so a timeout is needed to wait for the page    to load

* With beautiful soup accessing the html code and taking the name of the best 150 apps in the category: Games

* Finally, with google play scraper access play store and download the reviews


In [10]:
url='https://www.applyzer.com/?mmenu=worldcharts'     


# configure webdriver  -- headless in order not to open the webpage everytime
options = Options()
options.headless = True  # hide GUI
options.add_argument("--window-size=1920,1080")  # set window size to native GUI size
options.add_argument("start-maximized")  # ensure window is full-screen


# configure chrome browser to not load images to not waste space and time
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option(
    # this will disable image loading
    "prefs", {"profile.managed_default_content_settings.images": 2}
)


driver = webdriver.Chrome('/Users/MyUsername/Downloads/chromedriver',options = options, chrome_options=chrome_options)
driver.get(url)


# wait for page to load to get the apps names to load 
element = WebDriverWait(driver=driver, timeout=10).until(
   EC.presence_of_element_located((By.NAME, 'categoryid')))


select = Select(driver.find_element(By.NAME,"categoryid"))
select.select_by_value('21')  #setting the category to Games

#print(driver.page_source)


    

<ipython-input-10-5c12c8b20b01>:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/MyUsername/Downloads/chromedriver',options = options, chrome_options=chrome_options)
<ipython-input-10-5c12c8b20b01>:19: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('/Users/MyUsername/Downloads/chromedriver',options = options, chrome_options=chrome_options)


In [12]:
soup = BeautifulSoup(driver.page_source, "html.parser")
finding = soup.body.find(id="content").find(id="placeholder").select('div.charto a')


#creating a list with the scraped apps
top_list =[] #contains the names of every app (free app)
for i in list(range(1,300,3)) + list(range(301,400,2)):
    print(finding[i].text)
    top_list.append(finding[i].text)

Makeover Studio: Makeup GamesLIHAO TECH CO., LIMITED
FIFA Mobile: FIFA World Cup™EA Swiss Sarl, Electronic Arts
Radix AccountingInderjit Singh Rupal
Subway SurfersSybo Games ApS
Tap Away 3DPopcore GmbH
Soccer Super StarSIMFACE LIMITED, Real Free Soccer
RobloxRoblox Corporation
Radix AccountingInderjit Singh Rupal
TownshipPLR Worldwide Sales Limited, Playrix
Idle Pocket PlanetHyperBeard Inc.
Call of Duty®: MobileActivision Publishing, Inc.
Block Blast-Block Puzzle GamesZhenning Shunning Network Technology Co., Ltd., Shunning Casual
WordscapesPeopleFun, Inc.
Apex Legends MobileEA Swiss Sarl, Electronic Arts
GardenscapesPLR Worldwide Sales Limited, Playrix
Parking Jam 3DPopcore GmbH
My Singing MonstersBig Blue Bubble
Coin MasterMoon Active LTD, Moon Active
Fade Master 3D: Barber ShopALICTUS YAZILIM EGITIM BILISIM TEKNOLOJILERI ELEKTRONIK SANAYI VE TICARET A.S., Alictus
Wordle!Lion Studios Plus LLC, Lion Studios Plus
Royal MatchDream Games Teknoloji Anonim Sirketi, Dream Games
Tetris®Plays

In [ ]:
#creation of a datafrake
df = pd.DataFrame()
rev = {}

#for each name downloading the app id from play store
for x in top_list:
    name = search(x,
                lang="en",  # defaults to 'en'
                country="us",  # defaults to 'us'
                n_hits=1  # defaults to 30 (= Google's maximum)
    )
    
    #In the case the app is not found in play store, store a blank name 
    if name != []:
        id_name = name[0]['appId']
    else:
        id_name = ""
        continue

    print(x,id_name)
    
    
    #For each app download 5,3 and 1 star reviews (150 each)
    for i in [5,3,1]:
        result, continuation_token = reviews(
        id_name,
        lang='en', # defaults to 'en'
        country='us', # defaults to 'us'
        sort=Sort.MOST_RELEVANT, # defaults to Sort.NEWEST
        count=150, # defaults to 100
        filter_score_with=i
            # defaults to None(means all score)
        )
        if result != []:
            for srev in result:
                df = df.append({"rating":i,"review":srev["content"]},ignore_index=True)
                

#store the reviews inside a csv called "game_rating.csv"
#df.to_csv("game_rating.csv", sep='\t', encoding='utf-8')
   

In [ ]:
df
